In [ ]:
%pip install xmltodict

In [ ]:
#import CSV package for reading from and writing into the csv file
import csv
#import package json for parsing json file
import json
#Import package for Parsing the given XML input and convert it into a dictionary
import xmltodict

#open csv file
def readCSVFile(name_file):
    lst=[*csv.DictReader(open(name_file))]
    return lst


def readJsonFile(name_file):
    # Open JSON file
    f = open(name_file)

    # returns JSON object as
    # a dictionary
    data = json.load(f)

    # Closing file
    f.close()
    return data

#Open xml file
def readXMLFile(name_file):
    with open(name_file) as fd:
        doc = xmltodict.parse(fd.read(), process_namespaces=True)
        return doc
#write file
def writeFile(nameFile, data):
    listKey = []
    for d in data[1]:
            listKey.append(d)

    # listKey = set(listKey)
#Now first we should write the data into the csv file
#for this we need to create a csv object
    with open(nameFile, 'w',newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=listKey, extrasaction='ignore',restval="")
        writer.writeheader()  #write header in the csv file
        writer.writerows(data)


def processingDataJson(data):
    for d in data:
        if 'debt' in d:
            temp = d['debt']
            del d['debt']
            if 'amount' in temp:
                d['debt_amount'] = temp['amount']
                d['debt_time_period_years'] = temp['time_period_years']
            else:
                d['debt_amount'] = temp
                d['debt_time_period_years'] = ''


def Merge(dict_1, dict_2):
	result = dict_1 | dict_2
	return result

#Re-format the dictionary
def format(dict):
    listKey = []
    for key in dict.keys():
        if str(key.find('@'))=='-1':
            if str(key.find('_'))=='-1':
               listKey.append(("".join([(" " + i if i.isupper() else i) for i in key]).strip()).capitalize())
            else:
                listKey.append((key.replace('_', ' ')).capitalize())
        else:
            key = (key.replace('@', '')).capitalize()
            if str(key.find('_'))=='-1':
               listKey.append(("".join([(" " + i if i.isupper() else i) for i in key]).strip()).capitalize())
            else:
                listKey.append((key.replace('_', ' ')).capitalize())

    listValues = []
    for value in dict.values():
        listValues.append(value)

    new_dict = {k: v for k, v in zip(listKey, listValues)}
    return new_dict

def formatList(listDict):
    newList = []
    for dict in listDict:
        newList.append(format(dict))
    return newList



def mergeJsonCSV(dict1,dict2):     #merge json to csv
    data = []
    for data1 in dict2:

        for data2 in dict1:         #delete duplicate
            if data2['firstName'] == data1['First Name'] and data2['lastName'] == data1['Second Name'] and str(data2['age']) == str(data1['Age (Years)']):
                # print('data1: '+str(data1))
                # print('data2: '+str(data2))

                del data2['firstName']
                del data2['lastName']
                del data2['age']

                data3 = Merge(data1,data2)
                data.append(data3)

                dict1.remove(data2)
                # print('data: '+str(data))
                # format(data1)

    return data

def mergeJsonCsvXml(dict1, dict2):         #merge 3 files json, csv and XML
    data = []

    for data1 in dict1:
        for data2 in dict2:                 #delete duplicate
            if data1['First Name'] == data2['@firstName'] and data1['Second Name'] == data2['@lastName'] and str(
                    data1['Age (Years)']) == str(data2['@age']):
                # print('data1: '+str(data1))
                # print('data2: '+str(data2))

                del data2['@firstName']
                del data2['@lastName']
                del data2['@age']

                data3 = Merge(data1, data2)
                data.append(data3)

                dict2.remove(data2)
                # print('data: '+str(data))
                # format(data1)
                # return data

    return data


if __name__ =="__main__":
    data_json = readJsonFile('user_data.json')
    data_csv = readCSVFile('user_data.csv')
    # print(data_csv)

    data_xml = readXMLFile('user_data.xml')
    # print(data_xml)

    processingDataJson(data_json)

    data_json_csv = mergeJsonCSV(data_json,data_csv)
    data = mergeJsonCsvXml(data_json_csv,data_xml['users']['user'])
    # print(data[0])
    # print(format(data[0]))
    # print(formatList(data))
    data = formatList(data)
#write the combined data file
    writeFile('data.csv',data)









In [ ]:
%pip install pony

In [ ]:
from pony.orm import *

db = Database()

class User(db.Entity):  #Defining entities which are related to the Database object
    id = PrimaryKey(int, auto=True)   #auto create id and set id to be primary key
    name = Required(str)
    age = Required(int)
    sex = Required(str)
    dependants = Required(int)
    retired = Required(bool)
    pension = Required(int)
    marital_status = Required(str)
    salary = Required(int)
    address_city = Required(str)
    company = Required(str)
    address_postcode = Required(str)
    address_main = Required(str)
    commute_distance = Required(float)
    make = Required(str)
    model = Required(str)
    year = Required(int)
    type = Required(str)
    iban = Required(str)
    credit_card_number = Required(str)
    credit_card_security_code = Required(int)
    credit_card_start_date = Required(str)
    credit_card_end_date = Required(str)
    debt_time_period_years = Required(int)
    debt_amount = Required(float)



if __name__ =="__main__":   
    #Binding the database object to a specific database
    db.bind(provider='mysql', host='europa.ashley.work', user='student_bh49jh', passwd='iE93F2@8EhM@1zhD&u9M@K', db='student_bh49jh')

    db.generate_mapping(create_tables=True)

   #load data
    data = readCSVFile('data.csv')


    for d in data:
        # print(d)
        with db_session:
            p = User(name = d['First  name']+" "+d['Second  name'],
                     age = d['Age ( years)'],
                     sex =  d['Sex'],
                     dependants = int('-1' if d['Dependants']== '' else d['Dependants']),
                     retired = (d['Retired'] == 'True'),
                     pension = int(d['Pension']),
                     marital_status = d['Marital status'],
                     salary = int(d['Salary']),
                     address_city = d['Address city'],
                     company = d['Company'],
                     address_postcode = d['Address postcode'],
                     address_main = d['Address main'],
                     commute_distance = float(d['Commute distance']),
                     make=d['Vehicle  make'],
                     model=d['Vehicle  model'],
                     year=int(d['Vehicle  year']),
                     type=d['Vehicle  type'],
                     iban=d['Iban'],
                     credit_card_number=d['Credit card number'],
                     credit_card_security_code=int(d['Credit card security code']),
                     credit_card_start_date=d['Credit card start date'],
                     credit_card_end_date=d['Credit card end date'],
                     debt_time_period_years=int(
                         '0' if d['Debt time period years'] == '' else d['Debt time period years']),
                     debt_amount=float('0' if d['Debt amount'] == '' else d['Debt amount']),
                     )
            # commit() will be done automatically
            # database session cache will be cleared automatically
            # database connection will be returned to the pool